# Read the Code


## Symmetry

The information is saved to
```c++
uint32_t data;
```

A symmetry class has such properties

```c++
bool is_fermion()
// N and 2S must be of the same odd/even property (not checked)
int n()     // number of sites
int twos()  // spin*2
int pg()    // point group number
int multiplicity()  // spin*2 + 1
```

setting methods
```c++

```

### SZ

```c++
// (N: 14bits) - (2S: 14bits) - (fermion: 1bit) - (pg: 3bits)
    SZLong(int n, int twos, int pg)
        : data((((uint32_t)n >> 1) << 18) |
               ((uint32_t)((twos & 0x7FFFU) << 3) | pg)) {}
```

###SU2

```c++
// (2N: 10bits) - (SL: 9bits) - (S: 9bits) - (fermion: 1bit) - (pg: 3bits)
    SU2Long(int n, int twos_low, int twos, int pg)
        : data((uint32_t)(((n >> 1) << 22) | ((twos_low >> 1) << 13) |
                          (twos << 3) | pg)) {}
```

In [38]:
SZ = block2.SZ

symeg = SZ(10, 11, 2)    # SZLong
print(symeg)

print(bin(symeg.data))
print(symeg.pg)
print(symeg.n)
print(symeg.is_fermion)

other_symeg = SZ(4, 3, 0)
print(other_symeg)
print(symeg + other_symeg)

< N=11 SZ=11/2 PG=2 >
0b101000000000001011010
2
11
True
< N=5 SZ=3/2 PG=0 >
< N=16 SZ=7 PG=2 >


## StateInfo

``` cpp
struct StateInfo<S,
                 typename enable_if<integral_constant<
                     bool, sizeof(S) % sizeof(uint32_t) == 0>::value>::type> {
    shared_ptr<vector<uint32_t>> vdata;
    // Array for symmetry labels
    S *quanta;
    // Array for number of states
    ubond_t *n_states;
    int n;
    total_bond_t n_states_total;
```

### Allocation

``` cpp
    void allocate(int length, uint32_t *ptr = 0) {
        if (ptr == 0) {
            vdata = make_shared<vector<uint32_t>>(_SI_MEM_SIZE(length));
            ptr = vdata->data();
        }
        n = length;
        quanta = (S *)ptr;
        n_states = (ubond_t *)(ptr + length * (sizeof(S) >> 2));
    }
```

### `find_state`

For a symmetry element `q`, find its location index in `StateInfo::quanta`.

``` cpp

    int find_state(S q) const {
        auto p = lower_bound(quanta, quanta + n, q);
        if (p == quanta + n || *p != q)
            return -1;
        else
            return (int)(p - quanta);
    }
```

## GeneralHamiltonian

``` cpp
template <typename S, typename FL>
struct GeneralHamiltonian<S, FL, typename S::is_su2_t> : Hamiltonian<S, FL> {
```

### Attributes

``` cpp
    typedef typename GMatrix<FL>::FP FP;
    using Hamiltonian<S, FL>::vacuum;
    using Hamiltonian<S, FL>::n_sites;
    using Hamiltonian<S, FL>::basis;
    using Hamiltonian<S, FL>::site_op_infos;
    using Hamiltonian<S, FL>::orb_sym;
    using Hamiltonian<S, FL>::find_site_op_info;
    using Hamiltonian<S, FL>::opf;
    using Hamiltonian<S, FL>::delayed;
    // Sparse matrix representation for normal site operators
    vector<unordered_map<string, shared_ptr<SparseMatrix<S, FL>>>>
        site_norm_ops;
    struct pair_hasher {
        size_t
        operator()(const pair<typename S::pg_t, typename S::pg_t> &x) const {
            size_t r = x.first;
            r ^= x.second + 0x9e3779b9 + (r << 6) + (r >> 2);
            return r;
        }
    };
    // Primitives for sparse matrix representation for normal site operators
    unordered_map<pair<typename S::pg_t, typename S::pg_t>,
                  unordered_map<string, shared_ptr<SparseMatrix<S, FL>>>,
                  pair_hasher>
        op_prims;
    const static int max_n = 10, max_s = 10;
    int twos;
```

Initialization with `vacuum(symmetry), n_sites(int), orb_sym(vector of pg_t)`

``` cpp
    GeneralHamiltonian(
        S vacuum, int n_sites,
        const vector<typename S::pg_t> &orb_sym = vector<typename S::pg_t>(),
        int twos = -1)
        : Hamiltonian<S, FL>(vacuum, n_sites, orb_sym), twos(twos) {
```

Use subclass `Hamiltonian`. 

``` cpp
        opf = make_shared<OperatorFunctions<S, FL>>(make_shared<CG<S>>());
        basis.resize(n_sites);
        site_op_infos.resize(n_sites);
        site_norm_ops.resize(n_sites);
        for (uint16_t m = 0; m < n_sites; m++)
            basis[m] = get_site_basis(m);
        init_site_ops();
    }

```

### Initialization of `basis`

``` cpp
    virtual shared_ptr<StateInfo<S>> get_site_basis(uint16_t m) const {
        // alpha and beta orbitals can have different pg symmetries
        return orb_sym.size() != n_sites * 2
                   ? SiteBasis<S>::get(orb_sym[m])
                   : SiteBasis<S>::get(orb_sym[m], orb_sym[m + n_sites]);
    }

```

in which the basis information is get from `SiteBasis::get` function in `hamiltonian.hpp`

``` cpp

template <typename S> struct SiteBasis<S, typename S::is_sz_t> {
    static shared_ptr<StateInfo<S>> get(int isyma, int isymb = -1) {
        if (isymb == -1)
            isymb = isyma;
        shared_ptr<StateInfo<S>> b = make_shared<StateInfo<S>>();
        b->allocate(4);
        b->quanta[0] = S(0, 0, 0);
        b->quanta[1] = S(1, 1, isyma);
        b->quanta[2] = S(1, -1, isymb);
        b->quanta[3] = S(2, 0, S::pg_mul(isyma, isymb));
        b->n_states[0] = b->n_states[1] = b->n_states[2] = b->n_states[3] = 1;
        b->sort_states();
        return b;
    }
};
```

so the `basis[m]` gets the `StateInfo` object in `state_info.hpp`, where the site symmetry information is saved in `basis[m].quanta` and the number of states are recorded in `basis[m].n_states`.


### Initialization of `site_op_infos`

`site_op_infos` is defined in the father class `Hamiltonian`

``` cpp
    vector<vector<pair<S, shared_ptr<SparseMatrixInfo<S>>>>> site_op_infos;
```


Allocation of `site_op_infos` is done by the `init_site_ops()` above. 

first initialize two VectorAllcator `i_alloc` and `d_alloc`.

``` cpp
    void init_site_ops() {
        shared_ptr<VectorAllocator<uint32_t>> i_alloc =
            make_shared<VectorAllocator<uint32_t>>();
        shared_ptr<VectorAllocator<FP>> d_alloc =
            make_shared<VectorAllocator<FP>>();
        const int max_n_odd = max_n | 1, max_s_odd = max_s | 1;
        const int max_n_even = max_n_odd ^ 1, max_s_even = max_s_odd ^ 1;
``` 

assign the `SparseMatrixInfo` to the symmetry key $(N, S, I_m)$ and $(N, S, I^{-1}_m)$ for each site $m$;

``` cpp
        // site operator infos
        for (uint16_t m = 0; m < n_sites; m++) {
            map<S, shared_ptr<SparseMatrixInfo<S>>> info;
            info[vacuum] = nullptr;
            for (int n = -max_n_odd; n <= max_n_odd; n += 2)
                for (int s = 1; s <= max_s_odd; s += 2) {
                    info[S(n, s, orb_sym[m])] = nullptr;
                    info[S(n, s, S::pg_inv(orb_sym[m]))] = nullptr;
                }
```

assign $(N, S, [I_m \otimes I_m, I_m^{-1} \otimes I_m, I_m^{-1} \otimes I_m^{-1}])$

``` cpp
            for (int n = -max_n_even; n <= max_n_even; n += 2)
                for (int s = -max_s_even; s <= max_s_even; s += 2) {
                    info[S(n, s, S::pg_mul(orb_sym[m], orb_sym[m]))] = nullptr;
                    info[S(n, s,
                           S::pg_mul(orb_sym[m], S::pg_inv(orb_sym[m])))] =
                        nullptr;
                    info[S(n, s,
                           S::pg_mul(S::pg_inv(orb_sym[m]), orb_sym[m]))] =
                        nullptr;
                    info[S(n, s,
                           S::pg_mul(S::pg_inv(orb_sym[m]),
                                     S::pg_inv(orb_sym[m])))] = nullptr;
``` 

if the length of org_sym is twice of site numbers, add the info for $I_{m + N_s}$.


``` cpp
                    if (orb_sym.size() == n_sites * 2) {
                        info[S(n, s,
                               S::pg_mul(orb_sym[m], orb_sym[m + n_sites]))] =
                            nullptr;
                        info[S(n, s,
                               S::pg_mul(orb_sym[m],
                                         S::pg_inv(orb_sym[m + n_sites])))] =
                            nullptr;
                        info[S(n, s,
                               S::pg_mul(S::pg_inv(orb_sym[m]),
                                         orb_sym[m + n_sites]))] = nullptr;
                        info[S(n, s,
                               S::pg_mul(S::pg_inv(orb_sym[m]),
                                         S::pg_inv(orb_sym[m + n_sites])))] =
                            nullptr;
                        info[S(n, s,
                               S::pg_mul(orb_sym[m + n_sites],
                                         orb_sym[m + n_sites]))] = nullptr;
                        info[S(n, s,
                               S::pg_mul(orb_sym[m + n_sites],
                                         S::pg_inv(orb_sym[m + n_sites])))] =
                            nullptr;
                        info[S(n, s,
                               S::pg_mul(S::pg_inv(orb_sym[m + n_sites]),
                                         orb_sym[m + n_sites]))] = nullptr;
                        info[S(n, s,
                               S::pg_mul(S::pg_inv(orb_sym[m + n_sites]),
                                         S::pg_inv(orb_sym[m + n_sites])))] =
                            nullptr;
                    }
                }
```

then the values in `info` is allocated by the `SparseMatrixInfo::initialize()` function, using the symmetry information of `p` (for operators?) and symmetry information of site `basis[m]`.


``` cpp

            for (auto &p : info) {
                p.second = make_shared<SparseMatrixInfo<S>>(i_alloc);
                p.second->initialize(*basis[m], *basis[m], p.first,
                                     p.first.is_fermion());
            }
```

then the temporary `info` is copied to each `site_op_infos[m]`.


``` cpp
            site_op_infos[m] = vector<pair<S, shared_ptr<SparseMatrixInfo<S>>>>(
                info.begin(), info.end());
        }
```

recall the definition of `site_op_infos` in `hamiltonian.cpp`

``` cpp
    vector<vector<pair<S, shared_ptr<SparseMatrixInfo<S>>>>> site_op_infos;

```

> `site_op_infos` is a vector(site index) of vector(index of symmetry) of (`S`, `SparseMatrixInfo<S>`) pair.



### Initialization of `op_prims`

recall

``` c++
    unordered_map<pair<typename S::pg_t, typename S::pg_t>,
                  unordered_map<string, shared_ptr<SparseMatrix<S, FL>>>,
                  pair_hasher>
        op_prims;
```

allocate the index `ipg = (ipg + ipb)`

``` c++
        for (uint16_t m = 0; m < n_sites; m++) {
            typename S::pg_t ipga = orb_sym[m], ipgb = ipga;
            if (orb_sym.size() == n_sites * 2)
                ipgb = orb_sym[m + n_sites];
            pair<typename S::pg_t, typename S::pg_t> ipg =
                make_pair(ipga, ipgb);
            if (this->op_prims.count(ipg) == 0)
                this->op_prims[ipg] =
                    unordered_map<string, shared_ptr<SparseMatrix<S, FL>>>();
            else
                continue;
            unordered_map<string, shared_ptr<SparseMatrix<S, FL>>> &op_prims =
                this->op_prims.at(ipg);
```
`op_prims` first initialize the SparseMatrix by allocating the sparse matrix infos of basis set `m` and the symmetry of the operator string `t`. For instance, if `t = ""` (operator $I$), the corresponding $(\Delta N, \Delta S, \Delta I) = (0, 0, 0)$. And then, the symmetry blocks and matrix elements of the operator is saved to the `SparseMatrix`.

$\hat I$ 

``` cpp
            op_prims[""] = make_shared<SparseMatrix<S, FL>>(d_alloc);
            op_prims[""]->allocate(find_site_op_info(m, S(0, 0, 0)));
            (*op_prims[""])[S(0, 0, 0)](0, 0) = 1.0;
            (*op_prims[""])[S(1, 1, ipga)](0, 0) = 1.0;
            (*op_prims[""])[S(1, -1, ipgb)](0, 0) = 1.0;
            (*op_prims[""])[S(2, 0, S::pg_mul(ipga, ipgb))](0, 0) = 1.0;

```

of $\hat a^\dagger_\alpha$: $\Delta N = 1, \Delta S = 1/2, \Delta I = I^\alpha$

``` cpp

            op_prims["c"] = make_shared<SparseMatrix<S, FL>>(d_alloc);
            op_prims["c"]->allocate(find_site_op_info(m, S(1, 1, ipga)));
            (*op_prims["c"])[S(0, 0, 0)](0, 0) = 1.0;
            (*op_prims["c"])[S(1, -1, ipgb)](0, 0) = 1.0;
```

$\hat a_\alpha$

``` cpp

            op_prims["d"] = make_shared<SparseMatrix<S, FL>>(d_alloc);
            op_prims["d"]->allocate(
                find_site_op_info(m, S(-1, -1, S::pg_inv(ipga))));
            (*op_prims["d"])[S(1, 1, ipga)](0, 0) = 1.0;
            (*op_prims["d"])[S(2, 0, S::pg_mul(ipga, ipgb))](0, 0) = 1.0;

``` 

$\hat a^\dagger_\beta$

``` cpp
            op_prims["C"] = make_shared<SparseMatrix<S, FL>>(d_alloc);
            op_prims["C"]->allocate(find_site_op_info(m, S(1, -1, ipgb)));
            (*op_prims["C"])[S(0, 0, 0)](0, 0) = 1.0;
            (*op_prims["C"])[S(1, 1, ipga)](0, 0) = -1.0;

```

$\hat a_\beta$, $\Delta N = -1, \Delta S = - 1/2 , \Delta I = I_\beta^{-1}$

``` cpp
            op_prims["D"] = make_shared<SparseMatrix<S, FL>>(d_alloc);
            op_prims["D"]->allocate(
                find_site_op_info(m, S(-1, 1, S::pg_inv(ipgb))));
            (*op_prims["D"])[S(1, -1, ipgb)](0, 0) = 1.0;
            (*op_prims["D"])[S(2, 0, S::pg_mul(ipga, ipgb))](0, 0) = -1.0;
        }
```

### Initialization of `site_norm_operators`

``` cpp
    vector<unordered_map<string, shared_ptr<SparseMatrix<S, FL>>>>
        site_norm_ops;
```

`site_norm_ops[m][string] = SparseMatrix`

``` cpp
        // site norm operators
        const string stx[5] = {"", "c", "C", "d", "D"};
        for (uint16_t m = 0; m < n_sites; m++) {
            typename S::pg_t ipga = orb_sym[m], ipgb = ipga;
            if (orb_sym.size() == n_sites * 2)
                ipgb = orb_sym[m + n_sites];
            pair<typename S::pg_t, typename S::pg_t> ipg =
                make_pair(ipga, ipgb);
            for (auto t : stx) {
                site_norm_ops[m][t] = make_shared<SparseMatrix<S, FL>>(nullptr);
                site_norm_ops[m][t]->allocate(
                    find_site_op_info(m,
                                      op_prims.at(ipg)[t]->info->delta_quantum),
                    op_prims.at(ipg)[t]->data);
            }
        }
    }
```

in saving the `site_norm_ops` for site `m` and string `t`, find the corresponding delta quantum, and save the sparse matrix in `op_prims` to it.

So `op_prims` is only temorately used. MPO do not use `op_prims` at all.



### `Hamiltonian::find_site_op_info`

for the site `i`, find the **location** of  `SparseMatrixInfo` in the `site_op_infos[i]` which corresponds to the symmetry `q`.

``` c++
    shared_ptr<SparseMatrixInfo<S>> find_site_op_info(uint16_t i, S q) const {
        auto p = lower_bound(site_op_infos[i].begin(), site_op_infos[i].end(),
                             q, SparseMatrixInfo<S>::cmp_op_info);
        if (p == site_op_infos[i].end() || p->first != q)
            return nullptr;
        else
            return p->second;
    }
```

## SparseMatrix

``` c++
template <typename S, typename FL> struct SparseMatrix {
    typedef typename GMatrix<FL>::FP FP;
    static const int cpx_sz = sizeof(FL) / sizeof(FP);
    shared_ptr<Allocator<FP>> alloc;
    shared_ptr<SparseMatrixInfo<S>> info;
    FL *data;
    FL factor;
    size_t total_memory;
    SparseMatrix(const shared_ptr<Allocator<FP>> &alloc = nullptr)
        : info(nullptr), data(nullptr), factor(1.0), total_memory(0),
          alloc(alloc) {}
```

### Allocation of a `SparseMatrix`

``` c++
    virtual void allocate(const shared_ptr<SparseMatrixInfo<S>> &info,
                          FL *ptr = 0) {
        this->info = info;
        total_memory = info->get_total_memory();
        if (total_memory == 0)
            return;
        if (ptr == 0) {
            if (alloc == nullptr)
                alloc = dalloc_<FP>();
            data = (FL *)alloc->allocate(total_memory * cpx_sz);
            memset(data, 0, sizeof(FL) * total_memory);
        } else
            data = ptr;
    }
```

### Initialiaztion

``` c++
    virtual void allocate(const shared_ptr<SparseMatrixInfo<S>> &info,
                          FL *ptr = 0) {
        this->info = info;
        total_memory = info->get_total_memory();
        if (total_memory == 0)
            return;
        if (ptr == 0) {
            if (alloc == nullptr)
                alloc = dalloc_<FP>();
            data = (FL *)alloc->allocate(total_memory * cpx_sz);
            memset(data, 0, sizeof(FL) * total_memory);
        } else
            data = ptr;
    }

```

Analyse the structure of sparce matrix info

``` c++
(*op_prims[""])[S(0, 0, 0)](0, 0) = 1.0;
```

From the definition of oprator `[]` in `SparseMatrix`: 

``` cpp
    GMatrix<FL> operator[](S q) const { return (*this)[info->find_state(q)]; }
    GMatrix<FL> operator[](int idx) const {
        assert(idx != -1);
        return GMatrix<FL>(data + info->n_states_total[idx],
                           (int)info->n_states_bra[idx],
                           (int)info->n_states_ket[idx]);
    }
```

The symmetry index `(*op_prims[""])[S(0, 0, 0)]` returns a GMatrix element:

``` c++
    GMatrix<FL> operator[](S q) const { return (*this)[info->find_state(q)]; }
```

which is really a index of pointer address (int):

``` c++
    GMatrix<FL> operator[](int idx) const {
        assert(idx != -1);
        return GMatrix<FL>(data + info->n_states_total[idx],
                           (int)info->n_states_bra[idx],
                           (int)info->n_states_ket[idx]);
    }
```

GMatrix initialization is

``` c++
    GMatrix(float *data, MKL_INT m, MKL_INT n) : data(data), m(m), n(n) {}
```

the index of coordinate is realized by

``` c++
    float &operator()(MKL_INT i, MKL_INT j) const {
#ifdef _AGGRESSIVE_DEBUG
        assert((size_t)i * n + j < (size_t)m * n);
#endif
        return *(data + (size_t)i * n + j);
    }
```

which points to a float pointer `float *data;`.

If one wants to save a $(n \times n)$ dense matrix in Boson number space, we should edit the `n_states_bra` and `n_states_ket` in the `SparseMatrixInfo`.

## SparseMatrixInfo

``` c++
struct SparseMatrixInfo<
    S, typename enable_if<integral_constant<
           bool, sizeof(S) % sizeof(uint32_t) == 0>::value>::type> {
    shared_ptr<Allocator<uint32_t>> alloc;
    // Composite quantum number for row and column quanta
    S *quanta;
    ubond_t *n_states_bra, *n_states_ket;
    uint32_t *n_states_total;
    S delta_quantum;
    bool is_fermion;
    bool is_wavefunction;
    // Number of non-zero blocks
    int n;

```

The construction function only allocates a location

``` c++
    SparseMatrixInfo(const shared_ptr<Allocator<uint32_t>> &alloc = nullptr)
        : n(-1), cinfo(nullptr), alloc(alloc) {}
```

### Initialization

``` c++

    void initialize(const StateInfo<S> &bra, const StateInfo<S> &ket, S dq,
                    bool is_fermion, bool wfn = false) {
        this->is_fermion = is_fermion;
        this->is_wavefunction = wfn;
        delta_quantum = dq;
        vector<S> qs;
        qs.reserve(ket.n);
        for (int i = 0; i < ket.n; i++) {
            S q = wfn ? -ket.quanta[i] : ket.quanta[i];
            S bs = dq + q;
            for (int k = 0; k < bs.count(); k++)
                if (bra.find_state(bs[k]) != -1)
                    qs.push_back(dq.combine(bs[k], q));
        }
```

`qs` saves the bra quanta which one-to-one corresponds to the ket quanta.

``` c++
        n = (int)qs.size();
        allocate(n);
        if (n != 0) {
            memcpy(quanta, qs.data(), n * sizeof(S));
            sort(quanta, quanta + n);
            for (int i = 0; i < n; i++) {
                assert(ket.find_state(wfn ? -quanta[i].get_ket()
                                          : quanta[i].get_ket()) != -1);
                // assert whethetr the qs has invalid element (see symmetry.hpp)
                n_states_ket[i] = ket.n_states[ket.find_state(
                    wfn ? -quanta[i].get_ket() : quanta[i].get_ket())];
                n_states_bra[i] =
                    bra.n_states[bra.find_state(quanta[i].get_bra(dq))];
            }
            n_states_total[0] = 0;
            for (int i = 0; i < n - 1; i++)
                n_states_total[i + 1] =
                    n_states_total[i] +
                    (uint32_t)n_states_bra[i] * n_states_ket[i];
        }
    }
```

in which the `n_states_ket` and `n_states_bra` is initialized by the `n_states` of `StateInfo bra` and `StateInfo ket`.

The initialization of `StateInfo` can be shown above, in the definition of `SiteBasis` (here is the example of **SZ** symmetry):

``` c++

template <typename S> struct SiteBasis<S, typename S::is_sz_t> {
    static shared_ptr<StateInfo<S>> get(int isyma, int isymb = -1) {
        if (isymb == -1)
            isymb = isyma;
        shared_ptr<StateInfo<S>> b = make_shared<StateInfo<S>>();
        b->allocate(4);
        b->quanta[0] = S(0, 0, 0);
        b->quanta[1] = S(1, 1, isyma);
        b->quanta[2] = S(1, -1, isymb);
        b->quanta[3] = S(2, 0, S::pg_mul(isyma, isymb));
        b->n_states[0] = b->n_states[1] = b->n_states[2] = b->n_states[3] = 1;
        b->sort_states();
        return b;
    }
};
```

I here define the structure `SiteBasisBoson` in `hamiltonian.hpp`, which will take the `int nbcut` argument for cutoff of phonon numbers. Default is set as 8.

``` c++
template <typename S, typename = void> struct SiteBasisBoson;

template <typename S> struct SiteBasisBoson<S, typename S::is_sz_t> {
    static shared_ptr<StateInfo<S>> get(int isyma, int isymb = -1, int nbcut = 8) {
        if (isymb == -1)
            isymb = isyma;
        shared_ptr<StateInfo<S>> b = make_shared<StateInfo<S>>();
        b->allocate(4);
        b->quanta[0] = S(0, 0, 0);
        b->quanta[1] = S(1, 1, isyma);
        b->quanta[2] = S(1, -1, isymb);
        b->quanta[3] = S(2, 0, S::pg_mul(isyma, isymb));
        b->n_states[0] = b->n_states[1] = b->n_states[2] = b->n_states[3] = nbcut;
        b->sort_states();
        return b;
    } 
};
```

Then such infos will be saved in `basis` in `GeneralHamiltonian`, and they are saved in the `site_op_infos`. The `op_prims` then use the `SparseMatrixInfo` in `site_op_infos` to initialize the `GMatrix` structure for saving operator matrix elements.


For other strings that are not in the `site_norm_ops`, we use `get_site_string_ops` to append.

``` cpp
    void get_site_string_ops(
        uint16_t m,
        unordered_map<string, shared_ptr<SparseMatrix<S, FL>>> &ops) {
        shared_ptr<VectorAllocator<FP>> d_alloc =
            make_shared<VectorAllocator<FP>>();
        shared_ptr<SparseMatrix<S, FL>> tmp =
            make_shared<SparseMatrix<S, FL>>(d_alloc);
        for (auto &p : ops) {
            if (site_norm_ops[m].count(p.first))
                p.second = site_norm_ops[m].at(p.first);
            else {
                p.second = site_norm_ops[m].at(string(1, p.first[0]));
                for (size_t i = 1; i < p.first.length(); i++) {
                    S q = p.second->info->delta_quantum +
                          site_norm_ops[m]
                              .at(string(1, p.first[i]))
                              ->info->delta_quantum;
                    tmp = make_shared<SparseMatrix<S, FL>>(d_alloc);
                    tmp->allocate(find_site_op_info(m, q));
                    opf->product(0, p.second,
                                 site_norm_ops[m].at(string(1, p.first[i])),
                                 tmp);
                    p.second = tmp;
                }
                site_norm_ops[m][p.first] = p.second;
            }
        }
    }
```

In [5]:
import numpy as np
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

L = 8
N = 8
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

t = 1
U = 2

b = driver.expr_builder()

# hopping term
b.add_term("cd",
    np.array([[[i, i + 1], [i + 1, i]] for i in range(N - 1)]).flatten(),
    [-t] * 2 * (N - 1))
b.add_term("CD",
    np.array([[[i, i + 1], [i + 1, i]] for i in range(N - 1)]).flatten(),
    [-t] * 2 * (N - 1))

# onsite term
b.add_term("cdCD",
    np.array([[i, ] * 4 for i in range(N)]).flatten(),
    [U] * N)


In [6]:
b.data = b.finalize()

print(b.data.exprs)
print([x for x in b.data.indices])
print([x for x in b.data.data])

VectorString[cd, dc, CD, DC, cdCD]
[VectorUInt16[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7], VectorUInt16[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7], VectorUInt16[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7], VectorUInt16[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7], VectorUInt16[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7]]
[VectorDouble[-1, -1, -1, -1, -1, -1, -1], VectorDouble[1, 1, 1, 1, 1, 1, 1], VectorDouble[-1, -1, -1, -1, -1, -1, -1], VectorDouble[1, 1, 1, 1, 1, 1, 1], VectorDouble[2, 2, 2, 2, 2, 2, 2, 2]]


In [28]:
hamil = driver.ghamil

print(hamil.site_op_infos[0][21])


(< N=0 SZ=-1 PG=0 >, DQ=< N=0 SZ=-1 PG=0 > N=1 SIZE=1
BRA < N=1 SZ=-1/2 PG=0 > KET < N=1 SZ=1/2 PG=0 > [ 1x1 ]
)


AttributeError: 'block2.sz.GeneralHamiltonian' object has no attribute 'site_norm_ops'